# Cointegration & Pairs Trading

> **Goal** – build a market-neutral spread from **two non-stationary price series** (A, B) that *is* stationary, and trade its mean reversion.

---

## 1  Why Pairs Instead of Singles?

| Single-Asset Mean Reversion | Pairs Trading |
|-----------------------------|---------------|
| Not every price reverts to a mean. | Even if **A** and **B** each wander, a weighted combination  may revert. |
| Need intrinsic “pull” forces. | We *create* the pull by choosing the right hedge ratio $$\beta$$ (the “spring constant”). |

---

## 2  Key Concepts

| Term | Intuition |
|------|-----------|
| **Correlation** | Boats sail roughly in the same compass direction. |
| **Cointegration** | Boats are **tethered** by a spring ⇒ their *distance* oscillates around 0. |
| **Spread** |  $$\text{Spread}_t = P^A_t - \beta P^B_t$$ |
| **Error-Correction Model (ECM)** | Dynamic that pulls the spread back whenever it strays. |

---

## 3  Finding the Hedge Ratio β

### 3.1  CADF (Engle–Granger 2-Step) 

1. OLS for β  
2. ADF test the residuals → p < α ⇒ accept cointegration

### 3.2  Johansen Test (one-step, multivariate)

- Tests rank *r* of Π; *r* > 0 ⇒ at least one cointegrating vector.

---

## 4  Validate Stationarity

### Augmented Dickey–Fuller on the S
- Reject H₀ (unit root) when p < α ⇒ spread is mean-reverting.

### Other sanity checks

| Test | Mean-Reverting Range |
|------|---------------------|
| **Hurst H** | H < 0.5 |
| **Variance Ratio** | VR < 1 |

---

## 5  Trading Logic – Rolling Bollinger Bands
| Condition | Action |
|-----------|--------|
| S > UB    | **Short** spread (sell A, buy β B) |
| S < LB    | **Long** spread (buy A, sell β B) |
| S crosses μ | Close / flat |

*Tip* – re-estimate β with a rolling window to avoid look-ahead bias.


* **Regime shifts** – β & stationarity can break; monitor rolling p-values.  
* **Execution** – need simultaneous fills, borrow cost, slippage.  
* **Capital** – each leg ties up margin; size accordingly. rmula Sheet

| Description | Formula |
|-------------|---------|
| Spread | $$S_t = P^A_t - β\,P^B_t$$ |
| ECM (two-stock) | $$\Delta P^A_t = α_A S_{t-1} + ε^A_t$$<br>$$\Delta P^B_t = α_B S_{t-1} + ε^B_t$$ |
| Half-life | $$\text{HL} = -\ln2 / \hat θ$$ (θ from AR(1) on S) |
| Bollinger Bands | $$UB/LB = μ_L(S) \pm k σ_L(S)$$ |

---

# Kalman Filter Pairs Trading

## Kalman Filter and Pairs Trading

### 1  Why use a Kalman Filter?  
* A static hedge ratio **β** from one-shot OLS quickly becomes stale.  
* Kalman Filter treats the intercept *aₜ* and slope *bₜ* as **time-varying states** and updates them with every new price pair \((xₜ,yₜ)\).  
* Result: a living, walk-forward hedge ratio that automatically adapts to market drift and removes look-ahead bias.

### 2  State-space set-up  
| Component | Meaning | This trade (EWA ~ EWC) |
|-----------|---------|-------------------------|
| **State** $ \thetaₜ = (aₜ,bₜ)^\top $ | Intercept & slope | Start at (1,1) with loose covariance |
| **Transition** $ \thetaₜ = G\theta_{t-1}+wₜ $ | Random walk | $G = I$,  $wₜ \sim 𝒩(0,Q)$ |
| **Observation** $ yₜ = Fₜ\thetaₜ + vₜ $ | One price at a time | $Fₜ = [1,\;xₜ]$ |
| **Noise** | Measurement error (spread) | $vₜ \sim 𝒩(0,R)$ |

> *The measured error \(eₜ = yₜ-\hat yₜ\) **is** the spread, and its model variance gives us a rolling σ for Bollinger bands.*

### 3  Trading logic (Bollinger bands on the Kalman spread)
1. **Daily loop**  
   * Feed $(xₜ,yₜ)$ into the filter ⇒ get $aₜ,bₜ$.  
   * Compute spread $\hat eₜ = yₜ-(aₜ+bₜxₜ)$ and its variance $Sₜ$.
2. **Bands**:  
   * Center $\muₜ=\hat eₜ$ (Kalman mean)  
   * Width $\pm δ\sqrt{Sₜ}$   (δ≈2)  
3. **Signals**  
   * Spread > upper band → **short** spread (sell y, buy β x).  
   * Spread < lower band → **long** spread.  
   * Spread crosses center → **flatten**.

### 4  Empirical take-aways  
* 2004-2016: adaptive β performs as well as, or better than, fixed-β pairs trade.  
* ≥ 2017: macro relationship between Australia & Canada weakened ⇒ strategy degrades, showing **asset-selection matters more than filtering technique**.  
* Kalman Filter excels at *micro* parameter drift, but cannot rescue a pair whose economic cointegration has broken.

### 5  Key formulae  
* **Prediction:** $ \hat\theta_{t|t-1}=G\hat\theta_{t-1|t-1},\qquad P_{t|t-1}=GP_{t-1|t-1}G^\top+Q $  
* **Update:** $ Kₜ=P_{t|t-1}Fₜ^\top(FₜP_{t|t-1}Fₜ^\top+R)^{-1} $  
  $ \hat\theta_{t|t}=\hat\theta_{t|t-1}+Kₜ(yₜ-Fₜ\hat\theta_{t|t-1}) $  
  $ P_{t|t}=(I-KₜFₜ)P_{t|t-1} $

---

**Bottom line:** Use Kalman Filter when you believe *“the hedge ratio drifts, but the mean-reversion idea still holds.”* It keeps the trade responsive without peeking into the future.
